In [236]:
import pandas as pd
import os
import numpy as np
import random

In [237]:
train_data = pd.read_csv('../data/cases_train.csv')
location_data = pd.read_csv('../data/location.csv')
test_data = pd.read_csv('../data/cases_test.csv')




In [238]:
train_data.dtypes

age                        object
sex                        object
province                   object
country                    object
latitude                  float64
longitude                 float64
date_confirmation          object
additional_information     object
source                     object
outcome                    object
dtype: object

In [239]:
def transform_age(s):
    age_str = str(s)

    for i, ch in enumerate(age_str):
        if not ch.isdigit():
            if age_str[i+1:].startswith('m') or age_str[i+1:].startswith('M'):
                return 0
            else:
                return int(age_str[:i])
    return int(age_str)


age1 = transform_age("10+")
age2 = transform_age("10 - 25")
age3 = transform_age("10-25")
age4 = transform_age("10 month")
age5 = transform_age("10 months")
age6 = transform_age("10 Month")
print(age1)
print(age2)
print(age3)
print(age4)
print(age5)
print(age6)

10
10
10
0
0
0


In [240]:

# preprocess : clean the age column
train_data['age'] = train_data['age'].apply(lambda x: transform_age(x) if np.all(pd.notnull(x)) else x)



In [241]:
print("===== BEFORE FILLING MISSING VALUE === ")
train_data.isnull().sum().sort_values(ascending = False)
# drop some columns
DROP_COLUMNS = [ 'additional_information', 'source']
train_data.drop(DROP_COLUMNS, axis=1, inplace=True)

# use the average value as the prediction value
AVERAGE_COLUMNS = ['age']

for column in AVERAGE_COLUMNS:
    mean_val = train_data[column].mean()
    train_data[column].fillna(mean_val, inplace=True)


# fill sex columns using a random value
SEX_CHOICE = ["male", "female"]
SEX_WEIGHTS = [0.55, 0.45]
#train_data['sex'].fillna(, inplace=True)
def generate_sex():
    return random.choices(SEX_CHOICE, SEX_WEIGHTS)[0]
train_data['sex'] = train_data['sex'].apply(lambda x: x if np.all(pd.notnull(x)) else generate_sex())


===== BEFORE FILLING MISSING VALUE === 


In [242]:
# datetime
def transform_datetime(date_str):
    for i, ch in enumerate(date_str):
        if ch == '-':
            return date_str[:i]
    return date_str


date1 = transform_datetime("10.03.2020-13.03.2020")
print(date1)

10.03.2020


In [243]:
train_data['date_confirmation'] = train_data['date_confirmation'].apply(lambda x: transform_datetime(x) if np.all(pd.notnull(x)) else x)
train_data['date_confirmation'] = pd.DatetimeIndex(train_data['date_confirmation']).month


In [244]:
# check the result
DROPNA_COLUMNS = ['date_confirmation', 'country']

train_data.dropna(subset=DROPNA_COLUMNS, inplace=True)

In [245]:
# after dropna
train_data.isnull().sum().sort_values(ascending = False)

province             4082
outcome                 0
date_confirmation       0
longitude               0
latitude                0
country                 0
sex                     0
age                     0
dtype: int64

In [246]:
location_data.head(5)

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,Afghanistan,2020-09-20 04:22:56,33.93911,67.709953,38919,1437,32576,4906.0,Afghanistan,99.976005,3.692284
1,NaN,Albania,2020-09-20 04:22:56,41.15330,20.168300,12226,358,6888,4980.0,Albania,424.838418,2.928186
2,NaN,Algeria,2020-09-20 04:22:56,28.03390,1.659600,49623,1665,34923,13035.0,Algeria,113.162645,3.355299
3,NaN,Andorra,2020-09-20 04:22:56,42.50630,1.521800,1564,53,1164,347.0,Andorra,2024.202420,3.388747
4,NaN,Angola,2020-09-20 04:22:56,-11.20270,17.873900,3901,147,1445,2309.0,Angola,11.869312,3.768265


In [247]:
# aggregate the location dataset
AGG_MAP = {'Confirmed': 'sum',
           'Deaths': 'sum',
           'Recovered': 'sum',
           'Active': 'sum',
           'Incidence_Rate': 'mean',
           'Case-Fatality_Ratio' : 'mean'}
location_data = location_data.groupby(['Province_State', 'Country_Region']).agg(AGG_MAP)
location_data.reset_index()


,Province_State,Country_Region,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio
0,Abruzzo,Italy,4137,474,2984,679.0,315.421095,11.457578
1,Acre,Brazil,27061,648,24298,2115.0,3068.366716,2.394590
2,Adygea Republic,Russia,3775,31,3253,491.0,832.642222,0.821192
3,Aguascalientes,Mexico,6581,555,5783,243.0,458.722950,8.433369
4,Aichi,Japan,5077,79,4554,444.0,67.225097,1.556037
...,...,...,...,...,...,...,...,...
569,Zaporizhia Oblast,Ukraine,2829,42,1075,1712.0,165.842437,1.484624
570,Zeeland,Netherlands,1210,72,0,1138.0,315.524866,5.950413
571,Zhejiang,China,1282,1,1272,9.0,2.234617,0.078003
572,Zhytomyr Oblast,Ukraine,4931,90,2379,2462.0,404.116398,1.825188


In [248]:
# location_data.loc[location_data['Country_Region'] == 'US']


In [249]:
location_data.to_csv(path_or_buf='../data/aggregated_location.csv', index=True)
train_data.to_csv(path_or_buf='../data/clean_cases_train.csv', index=True)


In [250]:
# add a combination key column in train dataset
def generate_combined_key(row):
    if pd.isnull(row['province']):
        return row['country']
    else:
        return str(row['province']) + ', ' + row['country']
def generate_combined_key2(row):
    if pd.isnull(row['Province_State']):
        return row['Country_Region']
    else:
        return str(row['Province_State']) + ', ' + row['Country_Region']

train_data['Combined_Key'] = train_data.apply (lambda row: generate_combined_key(row), axis=1)
location_data['Combined_Key'] = location_data.apply (lambda row: generate_combined_key(row), axis=1)
# join the two dataset
train_data.to_csv(path_or_buf='../data/clean_cases_train_with_key.csv', index=False)

KeyError: 'province'

In [ ]:
# join the two  dataset

after_join = pd.merge(train_data, location_data, how='left', on=['Combined_Key'])
after_join.isnull().sum().sort_values(ascending = False)